# 01 — Build ICU-Stay Cohort (MIMIC-IV Demo)

**Project:** Early ICU Mortality Prediction Using Structured EHR Data  
**Dataset:** MIMIC-IV Clinical Database Demo (v2.2)

## Goal of this notebook
Create a **cohort table** where:
- **1 row = 1 ICU stay** (`stay_id`)
- **Label:** in-hospital mortality (`hospital_expire_flag`)
- **Prediction time:** `intime + 24 hours` (a common, leakage-safe setup)

## Inputs (CSV)
- `patients.csv`
- `admissions.csv`
- `icustays.csv`

> We will **not** use `labevents.csv` here. Feature engineering happens in the next notebook.

## Output
- `cohort_icustay_mortality.csv`


In [ ]:
# Setup
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

# Reproducibility / environment info
import sys, platform
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Pandas:", pd.__version__)


## 1) Locate the data folder

Set `DATA_DIR` to the folder containing the CSV files.

If you're running this notebook in the same folder as `patients.csv`, `admissions.csv`, and `icustays.csv`, you can leave it as `Path('.')`.


In [ ]:
# Choose the Current folder
DATA_DIR = Path(".")

print("Using DATA_DIR:", DATA_DIR.resolve())


## 2) Load the starting tables

These are the **spine tables** for a first MIMIC project:
- `patients` → who the patient is
- `admissions` → the hospital admission and mortality label
- `icustays` → ICU stay timing (our unit of analysis)

We’ll load them into pandas and do a few sanity checks.


In [ ]:
# Load tables
patients = pd.read_csv(DATA_DIR / "patients.csv")
admissions = pd.read_csv(DATA_DIR / "admissions.csv")
icustays = pd.read_csv(DATA_DIR / "icustays.csv")

print("Loaded tables:")
print(f"  patients:   {patients.shape}")
print(f"  admissions: {admissions.shape}")
print(f"  icustays:   {icustays.shape}")

display(patients.head(3))
display(admissions.head(3))
display(icustays.head(3))


## 3) Validate expected columns (join keys + times + label)

We check that the columns needed for cohorting exist:
- Join keys: `subject_id`, `hadm_id`, `stay_id`
- Times: `admittime`, `dischtime`, `intime`, `outtime`
- Label: `hospital_expire_flag`


In [ ]:
required_patients = {"subject_id"}
required_admissions = {"subject_id", "hadm_id", "admittime", "dischtime", "hospital_expire_flag"}
required_icustays = {"subject_id", "hadm_id", "stay_id", "intime", "outtime"}

missing_patients = required_patients - set(patients.columns)
missing_admissions = required_admissions - set(admissions.columns)
missing_icustays = required_icustays - set(icustays.columns)

assert not missing_patients, f"patients missing: {missing_patients}"
assert not missing_admissions, f"admissions missing: {missing_admissions}"
assert not missing_icustays, f"icustays missing: {missing_icustays}"

print("All required columns are present ✅")


## 4) Parse timestamps as datetimes

This is essential for leakage-safe prediction tasks (we’ll later filter features to occur **before** prediction time).


In [ ]:
# Parse date/time columns
admissions["admittime"] = pd.to_datetime(admissions["admittime"], errors="coerce")
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"], errors="coerce")

# Some versions include ED times; parse if present
for c in ["edregtime", "edouttime", "deathtime"]:
    if c in admissions.columns:
        admissions[c] = pd.to_datetime(admissions[c], errors="coerce")

icustays["intime"] = pd.to_datetime(icustays["intime"], errors="coerce")
icustays["outtime"] = pd.to_datetime(icustays["outtime"], errors="coerce")

admissions[["admittime", "dischtime"]].head()


## 5) Basic sanity checks

We’re not “cleaning” yet — just verifying:
- IDs are mostly non-null
- uniqueness looks sensible
- label distribution exists


In [ ]:
print("Null checks:")
print("  admissions.hadm_id null:", admissions["hadm_id"].isna().sum())
print("  icustays.stay_id null:", icustays["stay_id"].isna().sum())
print("  icustays.intime null:", icustays["intime"].isna().sum())
print("  icustays.outtime null:", icustays["outtime"].isna().sum())

print("\nUniqueness checks:")
print("  patients unique subject_id:", patients["subject_id"].nunique(), "rows:", len(patients))
print("  admissions unique hadm_id:", admissions["hadm_id"].nunique(), "rows:", len(admissions))
print("  icustays unique stay_id:", icustays["stay_id"].nunique(), "rows:", len(icustays))

print("\nMortality label distribution (admissions):")
display(admissions["hospital_expire_flag"].value_counts(dropna=False))


## 6) Build the cohort table (1 row = 1 ICU stay)

We join in this order:

`icustays` → `admissions` (to get the label + hospital context) → `patients` (demographics)

This gives us one row per ICU stay with the mortality label attached.


In [ ]:
cohort = (
    icustays
    .merge(admissions, on=["subject_id", "hadm_id"], how="inner", validate="many_to_one")
    .merge(patients, on="subject_id", how="inner", validate="many_to_one")
)

print("Cohort shape (rows = ICU stays):", cohort.shape)
display(cohort.head(5))


## 7) Define prediction time + label

**Prediction time:** 24 hours after ICU admission (`intime + 24h`)  
**Label:** `hospital_expire_flag` (1 = died in-hospital, 0 = survived)

> We are **not** using any features yet — this notebook only defines the study cohort.


In [ ]:
cohort["prediction_time"] = cohort["intime"] + pd.Timedelta(hours=24)
cohort["label_mortality"] = cohort["hospital_expire_flag"].astype("int64")

# Quick checks
print("Prediction time sample:")
display(cohort[["stay_id", "intime", "prediction_time"]].head())

print("\nLabel distribution (ICU stays):")
display(cohort["label_mortality"].value_counts())


## 8) Keep only the columns we need (cohort artifact)

We keep identifiers, ICU stay boundaries, prediction time, and the label.


In [ ]:
cohort_table = cohort[
    [
        "subject_id",
        "hadm_id",
        "stay_id",
        "intime",
        "outtime",
        "prediction_time",
        "label_mortality",
    ]
].copy()

# Optional: sort for readability
cohort_table = cohort_table.sort_values(["subject_id", "hadm_id", "stay_id"]).reset_index(drop=True)

display(cohort_table.head(10))
print("cohort_table rows:", len(cohort_table))


## 9) Save the cohort table

This CSV is the “contract” between notebooks:
- Notebook 01 defines the study population + label + prediction time
- Notebook 02 will read this file and build features **before** `prediction_time`


In [ ]:
OUTPUT_PATH = Path("cohort_icustay_mortality.csv")
cohort_table.to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH.resolve())
